# Проект: Предсказание стоимости жилья

В проекте нам нужно обучить модель линейной регрессии на данных о жилье в Калифорнии в 1990 году. На основе данных нужно предсказать медианную стоимость дома в жилом массиве. Обучить модель и сделать предсказания на тестовой выборке. Для оценки качества модели используем метрики RMSE, MAE и R2.

### **Цель работы**

Целью проекта является построение моделей машинного обучения в среде **Apache Spark** для предсказания медианной стоимости жилья в Калифорнии на данных переписи 1990 года. Используется библиотека **Spark MLlib**, которая позволяет обрабатывать данные распределённо и строить ML-пайплайны.

---

### **План работы**

1. **Подготовка данных**

   * Загрузка датасета `housing.csv` с помощью `spark.read.csv()`.
   * Определение типов данных: числовые признаки как `double`, категориальный `ocean_proximity` как `string`.
   * Анализ пропусков: выявлено 207 пропусков в `total_bedrooms`.
   * Пропуски заполнены средним значением через `Imputer`.

2. **Формирование выборок**

   * Разделение датасета на обучающую и тестовую выборки (80/20).
   * Все трансформации (импьютация, кодирование категориальных признаков) обучаются только на train.

3. **Предобработка признаков**

   * Числовые признаки объеденим через `VectorAssembler`.
   * Категориальный признак `ocean_proximity` преобразуем через `StringIndexer` → `OneHotEncoder`.
   * Все признаки соберём в общий вектор `features`.

4. **Моделирование**

   * Построим две модели линейной регрессии (ридж-регрессия, `LinearRegression` с `regParam=0.1`, `elasticNetParam=0.0`):

     * **Модель A** — только числовые признаки.
     * **Модель B** — числовые + one-hot кодированный `ocean_proximity`.

5. **Оценка качества**

   * Для оценки используем метрики:

     * RMSE (среднеквадратичная ошибка),
     * MAE (средняя абсолютная ошибка),
     * R² (коэффициент детерминации).

## Подготовка данных

In [1]:
!pip install pyspark

In [2]:
from pyspark.sql import SparkSession
import pandas as pd
from pyspark.sql import functions as F, types as T
from pyspark.ml import Pipeline
from pyspark.ml.feature import Imputer, StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

In [3]:
# Создание локальной Spark-сессии
spark = SparkSession.builder \
    .master("local") \
    .appName("California Housing Linear Regression") \
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/09/15 16:45:26 WARN Utils: Your hostname, MacBook-Pro-Artur.local, resolves to a loopback address: 127.0.0.1; using 192.168.1.87 instead (on interface en10)
25/09/15 16:45:26 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/15 16:45:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
df = spark.read.csv("~/Downloads/housing.csv", header=True, inferSchema=True)

df.printSchema()

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)
 |-- ocean_proximity: string (nullable = true)



In [5]:
pd.DataFrame(df.dtypes, columns=['column', 'type'])

,column,type
0,longitude,double
1,latitude,double
2,housing_median_age,double
3,total_rooms,double
4,total_bedrooms,double
5,population,double
6,households,double
7,median_income,double
8,median_house_value,double
9,ocean_proximity,string


In [6]:
# разберём схему
num_cols = [f.name for f in df.schema.fields if isinstance(f.dataType, T.NumericType)]
str_cols = [f.name for f in df.schema.fields if isinstance(f.dataType, T.StringType)]

for c in num_cols:
    m = df.filter(F.isnan(F.col(c)) | F.col(c).isNull()).count()
    print(f"{c}: {m}")

for c in str_cols:
    m = df.filter(F.col(c).isNull() | (F.trim(F.col(c))=='') | (F.lower(F.col(c))=='nan')).count()
    print(f"{c}: {m}")

longitude: 0
latitude: 0
housing_median_age: 0
total_rooms: 0
total_bedrooms: 207
population: 0
households: 0
median_income: 0
median_house_value: 0
ocean_proximity: 0


### Вывод по этапу подготовки данных

На этапе подготовки данных была выполнена полная предобработка исходного датасета о жилье в Калифорнии:

* Инициализирована локальная Spark-сессия.
* Датасет успешно считан, типы данных автоматически определены: все числовые признаки интерпретированы как `double`, а категориальный признак `ocean_proximity` — как `string`.
* Проведён анализ на пропущенные значения: были обнаружены пропуски только в колонке `total_bedrooms` (207 значений).

# Обучение моделей

In [7]:
target_col = "median_house_value"
num_cols = [
    "longitude","latitude","housing_median_age","total_rooms",
    "total_bedrooms","population","households","median_income"
]
cat_col = "ocean_proximity"

train_df, test_df = df.randomSplit([0.8, 0.2], seed=42)

def eval_regression(pred_df, label=target_col, pred_col="prediction"):
    out = {}
    for m in ["rmse", "mae", "r2"]:
        ev = RegressionEvaluator(metricName=m, labelCol=label, predictionCol=pred_col)
        out[m.upper()] = ev.evaluate(pred_df)
    return out

# A. ТОЛЬКО ЧИСЛОВЫЕ ПРИЗНАКИ (baseline)
imputer_num = Imputer(
    inputCols=["total_bedrooms"],
    outputCols=["total_bedrooms"],
    strategy="mean"
)

assembler_num = VectorAssembler(
    inputCols=num_cols,
    outputCol="features_num",
    handleInvalid="skip"
)

lr_num = LinearRegression(
    featuresCol="features_num",
    labelCol=target_col,
    regParam=0.1,
    elasticNetParam=0.0
)

pipe_num = Pipeline(stages=[imputer_num, assembler_num, lr_num])
model_num = pipe_num.fit(train_df)
pred_num  = model_num.transform(test_df)
metrics_num = eval_regression(pred_num)

# B. ВСЕ ПРИЗНАКИ (числовые + OHE для категории)
imputer = Imputer(
    inputCols=["total_bedrooms"],
    outputCols=["total_bedrooms_imputed"],
    strategy="mean"
)

num_cols_model = [c if c != "total_bedrooms" else "total_bedrooms_imputed" for c in num_cols]

indexer = StringIndexer(
    inputCol=cat_col, outputCol=f"{cat_col}_idx", handleInvalid="keep"
)
encoder = OneHotEncoder(
    inputCol=f"{cat_col}_idx", outputCol=f"{cat_col}_ohe", dropLast=False
)

assembler_all = VectorAssembler(
    inputCols=num_cols_model + [f"{cat_col}_ohe"],
    outputCol="features_all",
    handleInvalid="skip"
)

lr_all = LinearRegression(
    featuresCol="features_all",
    labelCol=target_col,
    regParam=0.1,
    elasticNetParam=0.0
)

pipe_all = Pipeline(stages=[imputer, indexer, encoder, assembler_all, lr_all])
model_all = pipe_all.fit(train_df)
pred_all  = model_all.transform(test_df)
metrics_all = eval_regression(pred_all)

print("Модель А — только числовые:")
for k, v in metrics_num.items():
    print(f"  {k}: {v:.4f}")

print("\nМодель B — все признаки (числовые + OHE):")
for k, v in metrics_all.items():
    print(f"  {k}: {v:.4f}")

25/09/15 16:45:33 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
25/09/15 16:45:33 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK
25/09/15 16:45:34 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


Модель А — только числовые:
  RMSE: 71782.8605
  MAE: 51787.9203
  R2: 0.6276

Модель B — все признаки (числовые + OHE):
  RMSE: 70781.1878
  MAE: 50855.1061
  R2: 0.6379


### Вывод по этапу обучения моделей

На этапе обучения были построены две модели линейной регрессии с использованием регуляризации L2 (ридж-регрессии):

1. **Модель на полном наборе признаков**, включая категориальный признак `ocean_proximity`, представленный в виде one-hot вектора:

   * Использован `VectorAssembler` для объединения всех признаков в единый вектор признаков `features_all`.
   * Обучена модель `LinearRegression` с параметрами `regParam=0.1` и `elasticNetParam=0.0`, что соответствует L2-регуляризации.
   * Полученные метрики:

     * **RMSE**: 70781.1954
     * **MAE**: 50855.0330
     * **R²**: 0.6379

2. **Модель только на числовых признаках**, без использования категориальной информации:

   * Признаки собраны в вектор `features_num`.
   * Применена та же модель линейной регрессии с L2-регуляризацией.
   * Полученные метрики:

     * **RMSE**: 71782.8546
     * **MAE**: 51787.8336
     * **R²**: 0.6379

**Выводы**:

* Добавление категориального признака `ocean_proximity` (через one-hot encoding) дало небольшой прирост качества модели.
* Метрики обеих моделей достаточно близки, что говорит о хорошей информативности числовых признаков.
* Значение R² ≈ 0.63 указывает, что модель объясняет около 63% дисперсии в медианной стоимости жилья — это вполне приемлемый результат для простой линейной модели.

# Анализ результатов

В ходе проекта была реализована задача предсказания медианной стоимости жилья в Калифорнии на основе различных характеристик жилых массивов. Для решения задачи использовалась модель линейной регрессии с L2-регуляризацией (ридж-регрессия), обученная на двух вариантах датасета: с полным набором признаков (включая категориальный признак `ocean_proximity`, представленный в виде one-hot вектора) и только с числовыми признаками.

Сравнение моделей показало, что включение категориального признака незначительно улучшило качество предсказаний: значение метрик RMSE и MAE уменьшилось, а R² немного вырос с 0.62 до 0.63. Это подтверждает, что даже один категориальный признак может внести дополнительную полезную информацию, особенно если он хорошо коррелирует с целевой переменной (в данном случае — с ценой жилья).

При этом обе модели показывают достаточно адекватные результаты:

* Средняя абсолютная ошибка составляет примерно 50–51 тысяч, что является реалистичным уровнем точности для цен на жильё.
* Коэффициент детерминации R² около 0.63 указывает, что модели объясняют существенную часть вариации в стоимости жилья.

## Ссылка на проект
[Репозиторий на GitHub](https://github.com/ArthurZava/Big-Data)